Python 3 明确区分了人类可读的文本字符串和原始的字节序列

Unicode 标准把字符标识和具体的字节表述进行了区分：
- 字符的标识，码位。十六进制表示，加前缀 `U+`
- 字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换时使用的算法

In [1]:
s = 'café'
len(s)

4

In [2]:
# bytes 字面量以 b 开头
b = s.encode('utf-8')
b

b'caf\xc3\xa9'

In [3]:
len(b)

5

In [4]:
b.decode('utf-8')

'café'

Python 3 内置两种基本的二进制序列类型
- bytes（不可变）
- bytearray（可变）

bytes 或 bytearray 对象的各个元素是 [0, 255] 之间整数，二进制序列的切片始终是同一类型的二进制序列

In [5]:
# bytes 对象可以从 str 对象用给定的编码构建
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [6]:
# 各个元素是 range(256) 内的整数
cafe[0]

99

In [7]:
# bytearray 对象没有字面量句法，以 bytearray() 和字节序列字面量参数的形式显示
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [8]:
cafe_arr[-1:]

bytearray(b'\xa9')

二进制序列是整数序列，但字面量表示法表明其中有 ASCII 文本，各个字节的值可能会使用下列三种不同的方式显示
- 可打印的 ASCII 范围内的字符，使用 ASCII 字符本身
- 制表符、换行符、回车符和 \ 对应的字节，使用转义序列 `\t`、`\n`、`\r`、`\`
- 其他字节的值，使用十六进制转义序列

除了格式化方法（format、format_map）和几个处理 Unicode 数据的方法（casefold、isdecimal、isidentifier、isnumeric、isprintable、encode）之外，str 类型的其他方法都支持 bytes 和 bytearray 类型。

如果正则表达式编译自二进制序列，re 模块中的正则表达式函数也能处理二进制序列。

In [9]:
# 解析十六进制数字对构成二进制序列（空格可选）
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

构建 bytes 或 bytearray 实例还可以调用各自的构造方法，传入下列参数
- 一个 str 对象和一个 encoding 关键字参数
- 一个可迭代对象，提供 0~255 之间的数值
- 一个整数，使用空字节创建对应长度的二进制序列
- 一个实现了缓冲协议的对象

使用缓冲类对象创建 bytes 或 bytearray 对象时，始终复制源对象中的字节序列
memoryview 对象允许在二进制数据结构之间共享内存
使用 struct 模块从二进制序列中提取结构化信息

In [10]:
import array

# 短整数（16位）
numbers = array.array('h', [-2, -1, 0, 1, 2])

# 使用数组中的原始数据初始化 bytes 对象
octets = bytes(numbers)

# 表示 5 个短整数的 10 个字节
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

struct 模块能处理 bytes、bytearray、memoryview 对象

memoryview 对象切片返回一个新的 memoryview 对象，而且不会复制字节序列

In [11]:
import struct

# <    小字节序
# 3s3s 两个 3 字节序列
# HH   两个 16 位二进制整数
fmt = '<3s3sHH'

with open('2077.gif', 'rb') as fp:
    # 使用文件内容创建一个 memoryview 对象
    img = memoryview(fp.read())
    
# 转换成字节序列
header = img[:10]
bytes(header)

b'GIF89a9\x000\x00'

In [12]:
# 拆包 memoryview 对象，得到类型、版本、宽度、高度
struct.unpack(fmt, header)

(b'GIF', b'89a', 57, 48)

In [13]:
del header  # 删除引用
del img     # 释放 memoryview 实例所占的内存

Python 自带了超过 100 种编解码器（codec，encoder/decoder），用于在文本和字节之间相互转换。

某些编码（ASCII、GB2312）不能表示所有的 Unicode 字符
UTF 编码的设计目的就是处理每一个 Unicode 码位

典型编码
- latin1（即 iso8859_1）
    - 其他编码（cp1252、Unicode）的基础，与 cp1252 的字节值/码位相同
- cp2152
    - Microsoft 制定的 latin1 超集
- cp437
    - IBM PC 最初的字符集
- gb2312
    - 用于编码简体中文的陈旧标准，亚洲语言中使用较广泛的多字节编码之一
- utf-8
    - Web 中最常见的 8 位编码，与 ASCII 兼容
- utf-16le
    - 所有 UTF-16 支持通过转义序列（成为代理对，surrogate pair）表示超过 U+FFFF 的码位

In [14]:
b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'.decode('utf_16')

'El Niño'

In [15]:
# 用 3 个编解码器得到字节序列
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


`utf_?` 编码能处理任何字符串

不是每一个字节都包含有效的 ASCII 字符，也不是每一个字符序列都是有效的 UTF-8 或 UTF-16

陈旧的 8 位编码（cp1252、iso8859_1、koi8_r）能解码任何字节序列流而不抛出错误

In [16]:
b'S\xc3\xa3o Paulo'.decode('utf_8')

'São Paulo'

In [17]:
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [18]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [19]:
# 无法处理 ã ，抛出异常
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [20]:
# 跳过无法编码的字符
city.encode('cp437', errors='ignore')

b'So Paulo'

In [21]:
# 把无法编码的字符替换成 `?`
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [22]:
# 把无法编码的字符替换成 XML 实体
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#227;o Paulo'

In [23]:
octets = b'Montr\xe9al'

# latin1 超集
octets.decode('cp1252')

'Montréal'

In [24]:
# 编码希腊文
octets.decode('iso8859_7')

'Montrιal'

In [25]:
# 编码俄文
octets.decode('koi8_r')

'MontrИal'

In [26]:
octets.decode('utf_8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [27]:
# � 官方指定的替换字符（REPLACEMENT CHARACTER），表示未知字符
octets.decode('utf_8', errors='replace')

'Montr�al'

BOM 即字节序标记（byte-order mark），指明编码时使用 Interl CPU 的小字节序

在小字节序设备中，各个码位的最低有效字节在前面

UTF-16 编码在要编码的文本前面加上特殊的不可见富足 ZERO WIDTH NO-BREAK SPACE（U+FEFE）
UTF-16 有两个变种，不会生成 BOM
- UTF-16LE，显式指明使用小字节序
- UTF-16BE，显式指明使用大字节序

UTF-8 的一大优势是，不管设备使用哪种字节序，生成的字节序列始终一致

In [28]:
# b'\xff\xfe' 即 BOM
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

In [29]:
# UTF-16 变种不会生成 BOM
u16le = 'El Niño'.encode('utf_16le')
u16le

b'E\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

In [30]:
u16be = 'El Niño'.encode('utf_16be')
u16be

b'\x00E\x00l\x00 \x00N\x00i\x00\xf1\x00o'

处理文件时应该始终显式指定编码

In [31]:
# 写入文件指定 UTF-8 编码
open('cafe.txt', 'w', encoding='utf_8').write('café')

4

In [32]:
# 读取文件使用区域设置中的默认编码
open('cafe.txt').read()

'caf茅'

In [33]:
# 默认情况下 open 函数采用文本模式，返回 TextIOWrapper 对象
fp = open('cafe.txt', 'w', encoding='utf_8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf_8'>

In [34]:
# 返回写入的 Unicode 字符数
fp.write('café')

4

In [35]:
fp.close()

import os

# 文件中有 5 个字节， é 占两个字节
os.stat('cafe.txt').st_size

5

In [36]:
# 没有显式指定编码
fp2 = open('cafe.txt')
fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp936'>

In [37]:
fp2.encoding

'cp936'

In [38]:
fp2.read()

'caf茅'

In [39]:
# 正确的编码打开
fp3 = open('cafe.txt', encoding='utf_8')
fp3

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf_8'>

In [40]:
fp3.read()

'café'

In [41]:
# 以二进制模式读取，返回 BufferReader 对象
fp4 = open('cafe.txt', 'rb')
fp4

<_io.BufferedReader name='cafe.txt'>

In [42]:
fp4.read()

b'caf\xc3\xa9'

Windows 中的各个默认编码值
- `locale.getpreferredencoding()` 是最重要的设置，文本文件默认使用该配置

In [43]:
import locale
import sys

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'cp936'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'cp936'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'gbk'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [44]:
# U+0301 是 COMBINING ACUTE ACCENT ，加在 e 后面得到 é
# 在 Unicode 标准中，é 和 e\u0301 这样的序列教标准等价物（canonical equivalent）
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [45]:
len(s1), len(s2)

(4, 5)

In [46]:
s1 == s2

False

In [47]:
# 规范化
from unicodedata import normalize

# NFC (Normalization Form C) 使用最少的码位构成等价的字符串
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [48]:
normalize('NFC', s1) == len(normalize('NFC', s2)

SyntaxError: unexpected EOF while parsing (<ipython-input-48-a808acb6e3ee>, line 1)

In [49]:
# NFD 把组合字符分解成基字符和单独的组合字符
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [50]:
normalize('NFD', s1) == len(normalize('NFD', s2))

False

In [51]:
# 使用 NFC 时，有些单字符会被规范成另一个单字符
from unicodedata import normalize, name

ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [52]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [53]:
ohm == ohm_c

False

In [54]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

NFKC 和 NFKD 的 K 表示 compatibility 兼容性，这两种是较为严格的规范化形式。各个兼容字符会被替换成一个或多个“兼容分解”字符。

理想情况下，格式化是外部标记的职责，不应该由 Unicode 处理。

使用 NFKC 和 NFKD 规范化形式时要小心，只能在特殊情况中使用，例如搜索和索引，而不能用于持久存储，因为这两种转换会导致数据丢失。

In [55]:
half = '½'
normalize('NFKC', half)

'1⁄2'

In [56]:
four_squared = '4²'
normalize('NFKC', four_squared)

'42'

In [57]:
micro = 'µ'
micro_kc = normalize('NFKC', micro)
micro, micro_kc

('µ', 'μ')

In [58]:
ord(micro), ord(micro_kc)

(181, 956)

In [59]:
name(micro), name(micro_kc)

('MICRO SIGN', 'GREEK SMALL LETTER MU')

对于只包含 latin1 字符的字符串 s，s.casefold() 得到的结果与 s.lower() 一样，除了：
- 微符号 µ 会变成小写的希腊字母 μ
- 德语 Eszett 会变成 ss

自 Python 3.4 起，str.casefold() 和 str.lower() 得到不同结果的有 116 个码位。

In [60]:
micro = 'µ'
name(micro)

'MICRO SIGN'

In [61]:
micro_cf = micro.casefold()
name(micro_cf)

'GREEK SMALL LETTER MU'

In [62]:
micro, micro_cf

('µ', 'μ')

In [63]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [64]:
eszett_cf = eszett.casefold()
eszett, eszett_cf

('ß', 'ss')

使用 NFC 和 NFD，为了合理比较 Unicode 字符串，大多数情况下 NFC 是最好的规范化形式。

In [65]:
# NFC 规范化
def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

# 大小写折叠
def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

In [66]:
nfc_equal('A', 'a')

False

In [67]:
fold_equal('A', 'a')

True

In [68]:
s1 = 'café'
s2 = 'cafe\u0301'

s1 == s2

False

In [69]:
nfc_equal(s1, s2)

True

In [70]:
fold_equal(s1, s2)

True

In [71]:
s3 = 'Straße'
s4 = 'strasse'

s3 == s4

False

In [72]:
nfc_equal(s3, s4)

False

In [73]:
fold_equal(s3, s4)

True

In [74]:
import unicodedata


def shave_marks(txt):
    """去掉全部变音符号"""
    
    # 分解为基字符和组合记号
    norm_txt = unicodedata.normalize('NFD', txt)
    
    # 过滤所有组合记号
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))
    
    # 重组所有字符
    return unicodedata.normalize('NFC', shaved)

In [75]:
# 只替换了 è ç í
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [76]:
# 替换了 έ é
greek = 'Ζέφυρος, Zéfiro'
shave_marks(greek)

'Ζεφυρος, Zefiro'

In [77]:
import string

def shave_marks_latin(txt):
    
    """把拉丁基字符中所有的变音符号删除"""
    
    # 分解为基字符和组合记号
    norm_txt = unicodedata.normalize('NFD', txt)
    
    latin_base = False
    keepers = []
    for c in norm_txt:
        # 基字符为拉丁字母时，跳过组合记号
        if unicodedata.combining(c) and latin_base:
            continue  # 忽略拉丁基字符上的变音符号
        # 否则，保存当前字符
        keepers.append(c)
        
        # 如果不是组合字符，那就是新的基字符，检测是否为拉丁字母
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    
    shaved = ''.join(keepers)
    
    # 重组所有字符
    return unicodedata.normalize('NFC', shaved)

In [78]:
shave_marks_latin(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [79]:
shave_marks_latin(greek)

'Ζέφυρος, Zefiro'

In [80]:
# 构建字符替换字符的映射表
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")

# 构建字符替换字符串的映射表
multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

# 合并映射表
multi_map.update(single_map)


def dewinize(txt):
    """把 Win1252 符号替换成 ASCII 字符或序列"""
    return txt.translate(multi_map)


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))
    
    # 德语 Eszett 替换成 ss
    no_marks = no_marks.replace('ß', 'ss')
    
    # 用 NFKC 规范化形式把字符和与之兼容的码位组合起来
    return unicodedata.normalize('NFKC', no_marks)

In [81]:
# 替换弯引号、项目符号、商标符号
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [82]:
# 去掉变音符号，替换德语 Eszett
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

In [83]:
dewinize(greek)

'Ζέφυρος, Zéfiro'

In [84]:
asciize(greek)

'Ζέφυρος, Zefiro'

对字符串来说，比较的是码位；在比较非 ASCII 字符时，稍微有些困难。

不同的区域采用的排序规则有所不同，葡萄牙语等很多语言按照拉丁字母表排序。

在 Python 中，非 ASCII 文本的标准排序方式是使用 `locale.strxfrm` 函数，该函数将会把字符串转换成适合所在区域进行比较的形式
- 使用 `locale.strxfrm` 函数做排序之前，要调用 `locale.setlocale(locale.LC_COLLATE, <<your_locale>>)`
- 区域设置是全局的，不推荐在库中调用 setlocale 函数
- 操作系统必须支持区域设置，否则 setlocale 函数会抛出异常
- 必须知道如何拼写区域名称

In [85]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [86]:
import locale

locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [87]:
sorted_fruits = sorted(fruits, key=locale.strxfrm)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [88]:
# Unicode 排序算法（Unicode Collation Algorithm, UCA）
import pyuca

coll = pyuca.Collator()
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

Unicode 标准库提供了一个完整的数据库，包括码位与字符名称之间的映射，还有各个字符的元数据，以及字符之间的关系。

re 模块对 Unicode 的支持并不充分；regex 模块的最终目的是取代 re 模块，以提供更好的 Unicode 支持。

In [89]:
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),                       # U+0000 格式的码位
          char.center(6),                             # 在长度为 6 的字符串中居中显示字符
          're_dig' if re_digit.match(char) else '-',  # 如果字符匹配正则表达式 r`\d`，显示 re_dig
          'isdig' if char.isdigit() else '-',         # 如果 char.isdigit() 返回 True，显示 isdig
          'isnum' if char.isnumeric() else '-',       # 如果 char.isnumeric() 返回 True，显示 isnum
          format(unicodedata.numeric(char), '5.2f'),  # 使用长度为 5、小数点后保留 2 位的浮点数显示数值
          unicodedata.name(char),                     # Unicode 标准中字符的名称
          sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


双模式 API ，即提供的函数能接受字符串或字节序列为参数，然后根据类型进行特殊处理。

如果使用字节序列构建正则表达式，\d 和 \w 等模式只能匹配 ASCII 字符；字符串模式就能匹配 ASCII 之外的 Unicode 数字或字母。

In [90]:
import re

# 字符串类型
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')

# 字节序列类型
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

# 泰米尔数字 1729
text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

# 字节序列只能用字节序列正则表达式搜索
text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # 字符串模式 r'\d+' 能匹配泰米尔数字和 ASCII 数字
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # 字节序列模式 rb'\d+' 只能匹配 ASCII 数字
print('Words')
print('  str  :', re_words_str.findall(text_str))        # 字符串模式 r'\w+' 能匹配字母、上标、泰米尔数字、ASCII 数字
print('  bytes:', re_words_bytes.findall(text_bytes))    # 字节序列模式 rb'\w+' 只能匹配 ASCII 字节中的字母和数字

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


如果必须处理/修正无法自动处理的文件名，可以把字节序列参数传给 os 模块中国的函数，得到字节序列返回值。这一特性允许处理任何文件名或路径名，不管里面有多少乱码（鬼符）。

os 模块提佛能够了特殊的编码和解码函数
- fsencode(filename)
    - 使用 `sys.getfilesystemencoding()` 返回的编解码器把 filename 编码成字节序列，否则返回未经修改的 filename 字节序列
- fsdecode(filename)
    - 使用 `sys.getfilesystemencoding()` 返回的编解码器把 filename 解码成字符串，否则返回未经修改的 filename 字符串
    
在 Unix 衍生平台中，这些函数使用 surrogateescape 错误处理方式，以避免遇到意外字节序列时卡住；Windows 使用的错误处理方式是 strict 。
- surrogateescape（PEP 383）处理意外字节序列或未知编码，把每个无法解码的字节替换成 Unicode 中 U+DC00~U+DCFF 之间的码位

In [91]:
import os

os.listdir('.')

['.ipynb_checkpoints',
 '0x01 数据模型.ipynb',
 '0x02 序列构成的数组.ipynb',
 '0x03 字典和集合.ipynb',
 '0x04 文本和字节序列.ipynb',
 '2077.gif',
 'cafe.txt',
 'digits-of-π.txt',
 'dummy',
 'floats.bin',
 'Romeo and Juliet.txt']

In [92]:
# 参数是字节序列
os.listdir(b'.')

[b'.ipynb_checkpoints',
 b'0x01 \xe6\x95\xb0\xe6\x8d\xae\xe6\xa8\xa1\xe5\x9e\x8b.ipynb',
 b'0x02 \xe5\xba\x8f\xe5\x88\x97\xe6\x9e\x84\xe6\x88\x90\xe7\x9a\x84\xe6\x95\xb0\xe7\xbb\x84.ipynb',
 b'0x03 \xe5\xad\x97\xe5\x85\xb8\xe5\x92\x8c\xe9\x9b\x86\xe5\x90\x88.ipynb',
 b'0x04 \xe6\x96\x87\xe6\x9c\xac\xe5\x92\x8c\xe5\xad\x97\xe8\x8a\x82\xe5\xba\x8f\xe5\x88\x97.ipynb',
 b'2077.gif',
 b'cafe.txt',
 b'digits-of-\xcf\x80.txt',
 b'dummy',
 b'floats.bin',
 b'Romeo and Juliet.txt']